In [5]:
import matplotlib.pyplot as plt
import os
import pandas as pd

In [ ]:
# Define the ranges for x and y
scenarios = range(1, 5)
attempts = range(0, 10)
types = ['confort','simple']
models = ['dqn', 'ppo']
# Create a figure for the plots
plt.figure(figsize=(15, 10))

df = pd.DataFrame(columns=['Scenario', 'Model', 'Attempt','Time Step' ,'Speed', 'Action', 'Type'])

# Iterate over the ranges and plot the values
for type in types:
    for model in models:
        for x in scenarios:
            for y in attempts:
                speed_path = f'test_results/speeds/speed_{x}_{model}_{type}_{y}.txt'
                action_path = f'test_results/actions/action_{x}_{model}_{type}_{y}.txt'
                speeds = []
                actions = []

                # Check if the file exists
                if os.path.exists(speed_path):
                    with open(speed_path, 'r') as file:
                        lines = file.readlines()
                        speeds = [float(line.strip()) for line in lines]
                if os.path.exists(action_path):
                    with open(action_path, 'r') as file:
                        lines = file.readlines()
                        actions = [line.strip() for line in lines]
                
                for i in range(len(speeds)):
                    df = pd.concat([df, pd.DataFrame([[x, model.upper(), y, i, speeds[i], actions[i], "Simple" if type == 'simple' else "Comfort"]], columns=['Scenario', 'Model', 'Attempt','Time Step' ,'Speed', 'Action', 'Type'])])

In [3]:
df

In [4]:
# Calculate average speed grouping by scenario model and time step
df_avg_speed = df.groupby(['Scenario', 'Model', 'Time Step','Type'])['Speed'].mean().reset_index()
df_min_speed = df.groupby(['Scenario', 'Model', 'Time Step','Type'])['Speed'].min().reset_index()
df_max_speed = df.groupby(['Scenario', 'Model', 'Time Step','Type'])['Speed'].max().reset_index()

# Plot the average speed and with shadow min and max
scenarios = df_avg_speed['Scenario'].unique()

# Create subplots
num_scenarios = len(scenarios)
fig, axes = plt.subplots(num_scenarios, 1, figsize=(10, 5 * num_scenarios), sharex=False)

# Plot the average speed with min and max shadows for each scenario
for i, scenario in enumerate(scenarios):
    ax = axes[i] if num_scenarios > 1 else axes
    for model in df_avg_speed['Model'].unique():
        for type in df_avg_speed['Type'].unique():
            df_scenario = df_avg_speed[(df_avg_speed['Scenario'] == scenario) & (df_avg_speed['Model'] == model) & (df_avg_speed['Type'] == type)]
            df_scenario_min = df_min_speed[(df_min_speed['Scenario'] == scenario) & (df_min_speed['Model'] == model) & (df_min_speed['Type'] == type)]
            df_scenario_max = df_max_speed[(df_max_speed['Scenario'] == scenario) & (df_max_speed['Model'] == model) & (df_max_speed['Type'] == type)]          
            ax.plot(df_scenario['Time Step'], df_scenario['Speed'], label=f'{model} {type}')
            ax.fill_between(df_scenario['Time Step'], df_scenario_min['Speed'], df_scenario_max['Speed'], alpha=0.2)
    
    ax.set_title(f'Scenario {scenario}')
    ax.set_xlabel('Time Step')
    ax.set_ylabel('Speed')
    ax.legend()
    ax.grid(True)

print(df.shape)

# Adjust layout and show the plot
plt.tight_layout()
plt.show()

In [ ]:
crash_path = 'test_results/crash.txt'
crash = []
if os.path.exists(crash_path):
    with open(crash_path, 'r') as file:
        lines = file.readlines()
        crash = [line.strip() for line in lines]

df_crash = pd.DataFrame(columns=['Scenario', 'Model', 'Attempt', 'Crash'])
for c in crash:
    scenario, model, type,attempt, crash = c.split('_')
    df_crash = pd.concat([df_crash, pd.DataFrame([[scenario, model, type, attempt, crash]], columns=['Scenario', 'Model', 'Type', 'Attempt', 'Crash'])])
    
df_crash


In [ ]:
# create table with comparison between ppo and dqn per scenario for Simple type, for the values of speed, crash and action
df_comparison = pd.DataFrame(columns=['Scenario', 'Model', 'Speed', 'Crash', 'Action'])
for scenario in scenarios:
    for model in models:
        df_scenario = df[(df['Scenario'] == scenario) & (df['Model'] == model) & (df['Type'] == 'Simple')]
        df_crash_scenario = df_crash[(df_crash['Scenario'] == str(scenario)) & (df_crash['Model'] == model)]
        df_comparison = pd.concat([df_comparison, pd.DataFrame([[scenario, model, df_scenario['Speed'].mean(), df_crash_scenario['Crash'].count(), df_scenario['Action'].mode()[0]]], columns=['Scenario', 'Model', 'Speed', 'Crash', 'Action'])])

df_comparison